# 上傳檔案到 NodeMCU (Upload files to NodeMCU) - MQTT

### 需先安裝 ampy ( Adafruit MicroPython Tool )
#### pip install adafruit-ampy
https://github.com/adafruit/ampy  

In [1]:
import os
import shutil

### 設定COM port (set current COM port)

In [2]:
com_port = 'COM7'
com_port = 'COM12'
com_port = 'COM13'
com_port = 'COM15'
# com_port = '/dev/ttyUSB0'

### Utility functions

In [3]:
def clear_local_folder(folder):
    print('\n[Clearing folder {}]'.format(folder))
    for file in os.listdir(folder):
        os.remove(os.path.join(folder, file))
        
        
def copy_one_file_to_local_folder(folder, file, destination_folder):
    print('Copying {} to {}'.format(file, destination_folder))
    shutil.copy(os.path.join(folder, file), destination_folder)
    
    
def copy_all_files_to_local_folder(folders, destination_folder):
    print('\n[Copying all files to upload folder {}]'.format(destination_folder))
    clear_local_folder(destination_folder)    
    
    for folder in folders: 
        for file in os.listdir(folder):
            if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_'):
                copy_one_file_to_local_folder(folder, file, destination_folder)

In [4]:
def list_files_in_device(com_port):
    files = !ampy --port {com_port} ls
    return sorted(files)


def cat_file_from_device(com_port, file):
    !ampy --port {com_port} get {file}
    

def delete_file_in_device(com_port, file):
    print('Deleting {}'.format(file))
    !ampy --port {com_port} rm {file}
    

def delete_files_in_device(com_port):
    print('\n[Deleting all files in device {}]'.format(com_port))
    for file in list_files_in_device(com_port):
        delete_file_in_device(com_port, file)
        
#     try:
#         !ampy --port {com_port} rmdir {'/'}
#     except Exception as e:
#         print(e)        
        
        
def copy_one_file_to_device(com_port, folder, file, mpy_only = False):
    if mpy_only:
        if file.endswith('.mpy'):
            print('Copying {}'.format(file))
            !ampy --port {com_port} put {os.path.join(folder, file)}  
    elif file.endswith('.py'):
        print('Copying {}'.format(file))
        !ampy --port {com_port} put {os.path.join(folder, file)}   
        

def delet_main_in_device(com_port, main_file_names):
    print('Deleting {}'.format(main_file_names))
    files = list_files_in_device(com_port)

    for file in main_file_names:
        if file in files:
            delete_file_in_device(com_port, file)
            
            
def delete_main_and_files_in_device(com_port, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    print('\n[Copying all files to device {}]'.format(com_port))
    delet_main_in_device(com_port, main_file_names)
    if mpy_only: delete_first = True
    if delete_first: delete_files_in_device(com_port)
        

def copy_all_files_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):    
    for file in os.listdir(folder):
        if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_') and not file in main_file_names:
            copy_one_file_to_device(com_port, folder, file, mpy_only)                    
    
    for file in main_file_names:
        if os.path.isfile(os.path.join(folder, file)):
            copy_one_file_to_device(com_port, folder, file, mpy_only)  
            
            
def do_all_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    delete_main_and_files_in_device(com_port, main_file_names, delete_first, mpy_only)
    copy_all_files_to_device(com_port, folder, main_file_names, delete_first, mpy_only)

### Copy *.py to device (all needed fils will be put in the same folder)

In [10]:
folders = [os.path.sep.join(['..', '..', 'codes', 'micropython']),
           os.path.sep.join(['..', '..', 'codes', 'micropython_mqtt']),
           os.path.sep.join(['..', '..', 'codes', 'node']),
           os.path.sep.join(['..', '..', 'codes', 'shared']),]
upload_folder = os.path.sep.join(['upload', 'py'])

copy_all_files_to_local_folder(folders, upload_folder)


[Copying all files to upload folder upload\py]

[Clearing folder upload\py]
Copying boot.py to upload\py
Copying hardware.py to upload\py
Copying led.py to upload\py
Copying main.py to upload\py
Copying u_python.py to upload\py
Copying u_python_profiler.py to upload\py
Copying watchdog.py to upload\py
Copying webrepl_cfg.py to upload\py
Copying robust.py to upload\py
Copying simple.py to upload\py
Copying asynch_result.py to upload\py
Copying mqtt_client_paho.py to upload\py
Copying mqtt_client_umqtt.py to upload\py
Copying node.py to upload\py
Copying phonebook.py to upload\py
Copying queue_manager.py to upload\py
Copying socket_client.py to upload\py
Copying worker.py to upload\py
Copying worker_config.py to upload\py
Copying worker_cpython.py to upload\py
Copying worker_neuron.py to upload\py
Copying worker_neuron_20170516.py to upload\py
Copying worker_upython.py to upload\py
Copying commander.py to upload\py
Copying config.py to upload\py
Copying config_local.py to upload\py
Copy

In [11]:
upload_folder_py = os.path.sep.join(['upload', 'py'])

do_all_to_device(com_port, upload_folder, delete_first = True)

print('\n[All done!]')


[Copying all files to device COM15]
Deleting ['main.py', 'main.mpy']

[Deleting all files in device COM15]
Deleting boot.py
Copying asynch_result.py
Copying boot.py
Copying commander.py
Copying config.py
Copying config_local.py
Copying data_transceiver.py
Copying hardware.py
Copying led.py
Copying mqtt_client_paho.py
Copying mqtt_client_umqtt.py
Copying node.py
Copying phonebook.py
Copying queue_manager.py
Copying robust.py
Copying simple.py
Copying socket_client.py
Copying u_python.py
Copying u_python_profiler.py
Copying watchdog.py
Copying webrepl_cfg.py
Copying worker.py
Copying worker_config.py
Copying worker_cpython.py
Copying worker_neuron.py
Copying worker_neuron_20170516.py
Copying worker_upython.py
Copying main.py

[All done!]


### Copy *.mpy to device

In [ ]:
upload_folder_py = os.path.sep.join(['upload', 'py'])
upload_folder_mpy = os.path.sep.join(['upload', 'mpy'])

do_all_to_device(com_port, upload_folder_mpy, mpy_only = True)

copy_one_file_to_device(com_port, upload_folder_py, 'boot.py')
copy_one_file_to_device(com_port, upload_folder_py, 'main.py')

print('\n[All done!]')

### 單一檔案上傳 (single file upload, in case needed)

In [15]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', '..', 'dmz']), 'config.py') 

Copying config.py


In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'node']), 'node.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'node']), 'mqtt_client_umqtt.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'node']), 'worker_config.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'node']), 'worker_neuron.py') 

In [10]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'shared']), 'config.py') 

Copying config.py


In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'shared']), 'commander.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'boot.py') 

In [8]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'main.py') 

Copying main.py


In [9]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'hardware.py') 

Copying hardware.py


### 列出檔案 (list files)

In [ ]:
# list_files_in_device(com_port)

### 檢查檔案內容 (check file content)

In [ ]:
# cat_file_from_device(com_port, 'boot.py')

In [9]:
# cat_file_from_device(com_port, 'config.py')

### 連網測試 (network config and test)

In [ ]:
# 連上網路
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('SSID','password');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('Wei RN4','51557010');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('Kingnet-70M-$370', '');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF);nic.ifconfig();nic.config('mac');nic.ifconfig((['mac',])

In [ ]:
# 發出 http request
# import socket;addr=socket.getaddrinfo('micropython.org',80)[0][-1]
# s = socket.socket();s.connect(addr);s.send(b'GET / HTTP/1.1\r\nHost: micropython.org\r\n\r\n');data = s.recv(1000);s.close()

In [ ]:
# Delete all files
# import u_python;u_python.del_all_files();import os;os.listdir()


### Run Broker container on Raspberry Pi
copy folder 'codes' to Raspberry Pi under folder '/data/elastic_network_of_things_with_micropython',  
so Raspberry Pi has folder '/data/elastic_network_of_things_with_micropython/codes'  
then run the command below on Raspberry Pi.

```
docker run -it -p 9662:9662 --name=Broker --hostname=Broker --volume=/data/elastic_network_of_things_with_micropython:/project wei1234c/python_armv7 /bin/sh -c "cd /project/codes/broker && python3 broker.py" 
```